<a href="https://colab.research.google.com/github/mmartigani/RoadMapIA/blob/main/LLAMA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Parameter Efficient Fine-tuning (QLoRA) de LLAMA
En este caso práctico, se propone al alumno la realización de Parameter Efficient Fine-tuning (PEFT)
sobre uno de los LLMs más potentes y populares de la actualidad,
su nombre es LLAMA 2 de la compañía Meta (anteriormente Facebook).
Para este ejercicio concreto se propone el uso de LLAMA-2-7b-chat
que es una versión de LLAMA-2 que tiene 7.000 millones de parámetros.
Concretamente, se propone un escenario en el que un periódico digital
quiere generar automáticamente el título de sus artículos,
la descripción y las palabras clave utilizadas para el SEO.
Un factor importante es que la generación de esta información no debe ser genérica,
debe adaptarse al estilo y tipo de títulos, descripciones y palabras clave que ha
generado el periódico para otros artículos pasados.

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 xformers

In [ ]:
En este ejercicio vamos a cargar el modelo de una manera especial.
Concretamente, vamos a aplicar una técnica denominada QLoRA (Quantized LoRA).
QLoRA es un enfoque de ajuste eficiente que reduce el uso de memoria lo suficiente
como para ajustar un modelo de 65B parámetros en una sola GPU de 48 GB, al tiempo que
conserva el rendimiento completo de la tarea de ajuste de 16 bits.
Un modelo cuantificado es un modelo que tiene sus parámetros/pesos en un tipo de datos
inferior al tipo de datos en el que fue entrenado. Por ejemplo, si se entrena con un tipo
de dato float de 32 bits y luego se convierten esos parámetros/pesos a un tipo de datos inferior,
como float de 16/8/4 bits, el efecto sobre el rendimiento del modelo es mínimo o nulo.
En este caso vamos a reducir la precisión de los parámetros del LLM a 4 bits. Esto ayuda a
disminuir enormemente el consumo de memoria del modelo, lo que facilita su manipulación y
ajuste (finetuning) en hardware más accesible, como una única GPU.
Repositorio GitHub: https://github.com/artidoro/qlora
Artículo: https://arxiv.org/abs/2305.14314

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch
# Definimos los paramétros para bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
# Leemos el modelo pre-entrenado el modelo LLAMA2-7b-chat
model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-2-7b-chat-hf",
    quantization_config=bnb_config,
    device_map={"": 0},
    # low_cpu_mem_usage=True # Reduccion del consumo de cpu y memoria al leer el modelo
)

model.config.use_cache = False
model.config.pretraining_tp = 1 # Un valor distinto de 1 activará el cálculo más preciso pero más lento de las capas lineales

In [ ]:
from transformers import AutoTokenizer
# Leemos el tokenizador
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [ ]:
from transformers import pipeline
# Creamos un pipeline para la tokenización y generación del texto
llama2_pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)

In [ ]:
prompt = """<<SYS>>
Comportate como un ChatBot amigable experto en programación en Python
<</SYS>>

Desarrolla un programa en Python que ordene una lista de 10 numeros enteros."""

In [ ]:
prompt = "Actúa como si fueses el mayor experto en historia del mundo. Describe \
en pocas palabras lo que ocurrió en la segunda guerra mundial."

In [ ]:
prompt_template = f"<s>[INST] {prompt} [/INST]"

# Invocamos el pipeline para realizar generación de texto
output = llama2_pipe(prompt_template)
print(output[0]['generated_text'].replace("[/INST]", "[/INST]\n\n"))

In [ ]:
prompt = """"La Revolución Industrial, que tuvo lugar principalmente en el siglo XIX, \
fue un período de grandes cambios tecnológicos, culturales y socioeconómicos que \
transformó a las sociedades agrarias en sociedades industriales. Durante este tiempo, \
hubo un cambio masivo de mano de obra de las granjas a las fábricas. Esto se debió a \
la invención de nuevas máquinas que podían realizar tareas más rápido y eficientemente \
que los humanos o los animales. Esta transición llevó a un aumento en la producción de \
bienes, pero también tuvo consecuencias negativas, como la explotación \laboral y la \
contaminación ambiental."""

In [ ]:
prompt_template = f"<s>[INST] <<SYS>>Dado un artículo de noticias, proporciona \
los siguientes campos en un diccionario JSON: \'titulo\', \'SEO\' y \'resumen\'\
<</SYS>> {prompt} [/INST]"

# Invocamos el pipeline para realizar generación de texto
output = llama2_pipe(prompt_template)
print(output[0]['generated_text'].replace("[/INST]", "[/INST]\n\n"))

In [ ]:
Al igual que para el resto de los LLMs, uno de los puntos más importantes de la selección y
preparación de los datos es el formato de los ejemplos de entrenamiento.
En el caso de Llama 2, los autores utilizaron la siguiente plantilla para los modelos de chat:

In [ ]:
<s>[INST] <<SYS>>
System prompt
<</SYS>>

User prompt [/INST] Model answer </s>

In [ ]:
<s>[INST] <<SYS>>Dado un artículo de noticias, proporciona los siguientes campos en un diccionario JSON: \'titulo\', \'SEO\' y \'resumen\'<</SYS>>  La empresa pública Canal de Isabel II congelará este 2019 las tarifas de agua por cuarto año consecutivo para los hogares de la región. Además, el Canal ha eliminado las órdenes de corte de suministro por impago a aquellas familias que no puedan abonar el suministro por problemas económicos, y extenderá bonificaciones a los perceptores de pensiones de viudedad con ingresos inferiores a los 14.000 euros brutos anuales. El bono social de Canal se aplica ya a los perceptores de la Renta Mínima de Inserción y de la Renta Activa de Inserción del Servicio Público de Empleo Estatal, pensiones no contributivas y, en general, a todas aquellas personas en situación de vulnerabilidad, además de familias numerosas. La Comunidad de Madrid volverá a congelar las tarifas del transporte público para este año 2019. Según lo recogido en los presupuestos regionales, los precios del abono transporte y los billetes sencillos para Metro, EMT y autobuses interurbanos se mantendrán en los mismos niveles que se fijaron en 2013. Se mantiene también la tarifa plana de 20 euros para el Abono de Transporte Joven, que ya cuenta con 1,3 millones de usuarios. Aparte, para este año se prevé la apertura de la nueva estación de Metro de Arroyo Fresno (L7) para el primer trimestre del año y que dará servicio a 220.000 vecinos de los barrios periféricos del noroeste de la capital. A ello se sumará también la reapertura de la \\"renovada\\" estación de Gran Vía, que servirá de intercambiador con la estación de Cercanías de la estación de Sol. También continuarán los trabajos para ampliar la Línea 11. A nivel estatal, el año 2019 se inicia con subidas en la luz, los carburantes, varios productos de telefonía, los sellos y los billetes regionales, mientras que bajará el gas natural, en una media del 1,92% el IBI y se mantendrán las tarifas de Cercanías y las tasas aeroportuarias [Qué sube y qué baja en 2019]. Sigue con nosotros la actualidad de Madrid en Facebook, en Twitter y en nuestro Patio de Vecinos en Instagram[/INST] "{\\"titulo\\": \\"Congelación de tarifas de agua y transporte público en Madrid para 2019\\", \\"SEO\\": [\\"Canal de Isabel II, tarifas de agua, transporte público, Madrid, 2019\\"], \\"resumen\\": \\"Canal de Isabel II congela tarifas de agua. Madrid mantiene precios de transporte público y anuncia mejoras en 2019.\\"}" </s>```

In [ ]:
# Leemos el conjunto de datos de Google Drive
dataset_path = "/content/drive/MyDrive/datasets/llama-2-fine-tuning-datset.txt"

with open(dataset_path, 'r', encoding='utf-8') as f:
  examples = f.read().splitlines() # De esta forma no sale el \n

In [ ]:
from datasets import Dataset, DatasetDict
ds = Dataset.from_dict({"text": examples})

In [ ]:
ds

In [ ]:
ds["text"][2]

In [ ]:
La siguiente función es interesante para comparar el número de parámetros
entrenables que tiene el modelo antes y después de apalicar LoRA

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [ ]:
print_trainable_parameters(model)

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training

# Definición de la configuración de LoRA
lora_config = LoraConfig(
                 r = 16, # Dimensión de las matrices
                 lora_alpha = 16, # LoRA scaling factor
                 lora_dropout = 0.05, # Regularización
                 bias="none",
                 task_type="CAUSAL_LM" # Tipo de tarea/modelo al que aplicarlo
                 # target_modules=["q", "k", "v"], # Los módulos (ej. Attention Heads) donde aplicar las matrices
)

In [ ]:
# Aplicamos la configuración al modelo
model_peft = get_peft_model(model, lora_config)

# Mostramos el número de parámetros que se van a entrenar
model_peft.print_trainable_parameters()

In [ ]:
# Directorio de salida donde se almacenarán las predicciones del modelo y los puntos de control
output_dir = "/content/drive/MyDrive/llama2-7b-chat-peft"

# Número de epochs de entrenamiento
num_train_epochs = 5

# Habilitar entrenamiento fp16/bf16 (establecer bf16 en True con un A100)
fp16 = False
bf16 = False

# Tamaño del lote por GPU para el entrenamiento
per_device_train_batch_size = 4

# Tamaño del lote por GPU para la evaluación
per_device_eval_batch_size = 4

# Número de pasos de actualización para acumular los gradientes
gradient_accumulation_steps = 1

# Habilitar el registro de puntos de control de gradientes
gradient_checkpointing = True

# Norma máxima del gradiente (recorte del gradiente)
max_grad_norm = 0.3

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    remove_unused_columns=False,
    save_strategy="epoch",
    save_total_limit=2
)

# Creamos la instancia de entrenamiento
trainer = SFTTrainer(
    model=model,
    train_dataset=ds,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=None, # Cuando es None, el max_seq_len vendrá determinado por la secuencia más larga de un lote
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False, # Empaquetar múltiples ejemplos cortos en la misma secuencia de entrada para aumentar la eficiencia
)

In [ ]:
# Iniciamos el entrenamiento
trainer.train()

In [ ]:
# Guardamos el tokenizador en disco para utilizarlo posteriormente
tokenizer.save_pretrained(f"{output_dir}/tokenizer")

In [ ]:
from transformers import pipeline
# Creamos un pipeline para la tokenización y generación del texto
llama2_ft_pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)

In [ ]:
prompt = """"La Revolución Industrial, que tuvo lugar principalmente en el siglo XIX, \
fue un período de grandes cambios tecnológicos, culturales y socioeconómicos que \
transformó a las sociedades agrarias en sociedades industriales. Durante este tiempo, \
hubo un cambio masivo de mano de obra de las granjas a las fábricas. Esto se debió a \
la invención de nuevas máquinas que podían realizar tareas más rápido y eficientemente \
que los humanos o los animales. Esta transición llevó a un aumento en la producción de \
bienes, pero también tuvo consecuencias negativas, como la explotación \laboral y la \
contaminación ambiental."""

In [ ]:
prompt_template = f"<s>[INST] <<SYS>>Dado un artículo de noticias, proporciona \
los siguientes campos en un diccionario JSON: \'titulo\', \'SEO\' y \'resumen\'\
<</SYS>> {prompt} [/INST]"

# Invocamos el pipeline para realizar generación de texto
output = llama2_ft_pipe(prompt_template)
print(output[0]['generated_text'].replace("[/INST]", "[/INST]\n\n"))

In [ ]:
!pip install sentencepiece

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM

model_name = "NousResearch/Llama-2-7b-chat-hf"
adapters_name = "/content/drive/MyDrive/llama2-7b-chat-peft/checkpoint-15"

print(f"Cargando el modelo: '{model_name}' en memoria...")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)

model = PeftModel.from_pretrained(model, adapters_name)
model = model.merge_and_unload()

print(f"El modelo: '{model_name}' ha sido cargado correctamente")

In [ ]:
from transformers import AutoTokenizer

# Leemos el tokenizador
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/llama2-7b-chat-peft/tokenizer", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [ ]:
from transformers import pipeline

# Creamos un pipeline para la tokenización y generación del texto
llama2_ft_pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)

In [ ]:
prompt = """"La Revolución Industrial, que tuvo lugar principalmente en el siglo XIX, \
fue un período de grandes cambios tecnológicos, culturales y socioeconómicos que \
transformó a las sociedades agrarias en sociedades industriales. Durante este tiempo, \
hubo un cambio masivo de mano de obra de las granjas a las fábricas. Esto se debió a \
la invención de nuevas máquinas que podían realizar tareas más rápido y eficientemente \
que los humanos o los animales. Esta transición llevó a un aumento en la producción de \
bienes, pero también tuvo consecuencias negativas, como la explotación \laboral y la \
contaminación ambiental."""

In [ ]:
prompt_template = f"<s>[INST] <<SYS>>Dado un artículo de noticias, proporciona \
los siguientes campos en un diccionario JSON: \'titulo\', \'SEO\' y \'resumen\'\
<</SYS>> {prompt} [/INST]"

# Invocamos el pipeline para realizar generación de texto
output = llama2_ft_pipe(prompt_template)
print(output[0]['generated_text'].replace("[/INST]", "[/INST]\n\n"))